In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/us-consumer-finance-complaints/database.sqlite
/kaggle/input/us-consumer-finance-complaints/consumer_complaints.csv


In [4]:
df = pd.read_csv('/kaggle/input/us-consumer-finance-complaints/consumer_complaints.csv')
df = df[['product','consumer_complaint_narrative']].dropna()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df.loc[df['product'] == 'Credit reporting, credit repair services, or other personal consumer reports', 'product'] = 'Credit reporting'
df.loc[df['product'] == 'Credit card or prepaid card', 'product'] = 'Credit card'
df.loc[df['product'] == 'Payday loan, title loan, or personal loan', 'product'] = 'Payday loan'
df.loc[df['product'] == 'Money transfer, virtual currency, or money service', 'product'] = 'Virtual currency'
df = df[(df['product'] != 'Other financial service')]

In [6]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].apply(clean_text)
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].str.replace('\d+', '')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  from ipykernel import kernelapp as app


In [7]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['consumer_complaint_narrative'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 56726 unique tokens.


In [8]:
X = tokenizer.texts_to_sequences(df['consumer_complaint_narrative'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (66696, 250)


In [9]:
Y = pd.get_dummies(df['product']).astype(np.float32).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (66696, 10)


In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(60026, 250) (60026, 10)
(6670, 250) (6670, 10)


In [12]:
class CustomMLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):       
        element_wise = tf.math.multiply_no_nan(x=y_true, 
                                                   y=tf.math.divide_no_nan(x=tf.math.subtract(x=y_true, y=y_pred), y=y_pred))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CustomMFullLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):       
        element_wise = tf.math.add(x=tf.math.divide_no_nan(x=tf.math.subtract(x=y_true, y=y_pred), y=y_pred),
                           y=tf.math.divide_no_nan(x=tf.math.subtract(x=1.0, y=y_true), y=tf.math.subtract(x=1.0, y=y_pred)))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CustomLLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):       
        element_wise = tf.math.divide_no_nan(x=y_true, 
                                             y=tf.math.sqrt(
                                                 x=tf.math.subtract(x=1.0, 
                                                                    y=tf.math.squared_difference(x=y_pred, y=1.0))
                                             ))
        
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CustomLFullLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):       
        element_wise = tf.math.subtract(x=tf.math.add(x=tf.math.divide_no_nan(x=y_true, 
                                             y=tf.math.sqrt(
                                                 x=tf.math.subtract(x=1.0, 
                                                                    y=tf.math.squared_difference(x=y_pred, y=1.0))
                                             )),
                                                      y=tf.math.divide_no_nan(x=tf.math.subtract(x=1.0,y=y_true),
                                                                              y=tf.math.sqrt(
                                                                                  tf.math.subtract(x=1.0,y=tf.math.square(y_pred))
                                                                              ))),
                                        y=1.0)
        
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))


class My_CE_Loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):        
        log_y_pred = tf.math.log(y_pred)
        element_wise = -tf.math.multiply_no_nan(x=log_y_pred, y=y_true)
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

In [37]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

2022-12-01 13:15:14.523799: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2022-12-01 13:15:14.523862: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:30019}
2022-12-01 13:15:14.529811: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2022-12-01 13:15:14.530049: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:30019}


## categorical_crossentropy function

In [54]:
with tpu_strategy.scope():
    model = Sequential()
    model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss=CustomLLoss(), optimizer='adam', metrics=['accuracy'])

In [55]:
BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync

In [56]:
epochs = 50
#Y_train1 = tf.convert_to_tensor(Y_train, dtype=tf.float32)
#Y_test1 = tf.convert_to_tensor(Y_test, dtype=tf.float32)
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=BATCH_SIZE,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/50
423/423 [==============================] - 24s 40ms/step - loss: nan - accuracy: 0.4778 - val_loss: 1.3562 - val_accuracy: 0.7748
Epoch 2/50
423/423 [==============================] - 13s 31ms/step - loss: nan - accuracy: 0.7907 - val_loss: 1.3362 - val_accuracy: 0.7751
Epoch 3/50
423/423 [==============================] - 13s 30ms/step - loss: nan - accuracy: 0.8163 - val_loss: 1.2926 - val_accuracy: 0.8031
Epoch 4/50
423/423 [==============================] - 13s 30ms/step - loss: nan - accuracy: 0.8554 - val_loss: 1.3020 - val_accuracy: 0.8088
Epoch 5/50
423/423 [==============================] - 13s 30ms/step - loss: nan - accuracy: 0.8621 - val_loss: 1.3458 - val_accuracy: 0.7868
Epoch 6/50
423/423 [==============================] - 13s 30ms/step - loss: nan - accuracy: 0.8804 - val_loss: 1.2909 - val_accuracy: 0.8273
Epoch 7/50
423/423 [==============================] - 13s 30ms/step - loss: nan - accuracy: 0.8829 - val_loss: 1.2945 - val_accuracy: 0.8219
Epoch 8/50
42

In [58]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

209/209 [==============================] - 4s 16ms/step - loss: nan - accuracy: 0.8381
Test set
  Loss: nan
  Accuracy: 0.838


## My_CE_Loss function

In [15]:
'''
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss=My_CE_Loss(), optimizer='adam', metrics=['accuracy'])
epochs = 5
batch_size = 64
'''
Y_train1 = tf.convert_to_tensor(Y_train, dtype=tf.float32)
Y_test1 = tf.convert_to_tensor(Y_test, dtype=tf.float32)
history = model.fit(X_train, Y_train1, epochs=epochs, batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

NameError: name 'batch_size' is not defined

In [43]:
accr = model.evaluate(X_test,Y_test1)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

209/209 [==============================] - 12s 58ms/step - loss: 0.5937 - accuracy: 0.8222
Test set
  Loss: 0.594
  Accuracy: 0.822


## CustomMLoss

In [40]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss=CustomMLoss(), optimizer='adam', metrics=['accuracy'])
epochs = 5
batch_size = 64
Y_train1 = tf.convert_to_tensor(Y_train, dtype=tf.float32)
Y_test1 = tf.convert_to_tensor(Y_test, dtype=tf.float32)
history = model.fit(X_train, Y_train1, epochs=epochs, batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
845/845 [==============================] - 919s 1s/step - loss: 3.6500 - accuracy: 0.6666 - val_loss: 2.5060 - val_accuracy: 0.7793
Epoch 2/5
845/845 [==============================] - 917s 1s/step - loss: 2.3062 - accuracy: 0.7927 - val_loss: 2.4146 - val_accuracy: 0.7884
Epoch 3/5
845/845 [==============================] - 914s 1s/step - loss: 1.7770 - accuracy: 0.8348 - val_loss: 2.5791 - val_accuracy: 0.7898
Epoch 4/5
845/845 [==============================] - 915s 1s/step - loss: 1.4690 - accuracy: 0.8520 - val_loss: 2.5211 - val_accuracy: 0.7878
Epoch 5/5
845/845 [==============================] - 910s 1s/step - loss: 1.2237 - accuracy: 0.8743 - val_loss: 2.3924 - val_accuracy: 0.8158


In [41]:
accr = model.evaluate(X_test,Y_test1)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

209/209 [==============================] - 12s 58ms/step - loss: 2.4192 - accuracy: 0.8270
Test set
  Loss: 2.419
  Accuracy: 0.827


## CustomMFullLoss

In [33]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss=CustomMFullLoss(), optimizer='adam', metrics=['accuracy'])
epochs = 5
batch_size = 64
Y_train1 = tf.convert_to_tensor(Y_train, dtype=tf.float32)
Y_test1 = tf.convert_to_tensor(Y_test, dtype=tf.float32)
history = model.fit(X_train, Y_train1, epochs=epochs, batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
845/845 [==============================] - 936s 1s/step - loss: 4.4119 - accuracy: 0.6741 - val_loss: 3.2997 - val_accuracy: 0.7851
Epoch 2/5
845/845 [==============================] - 928s 1s/step - loss: 3.1559 - accuracy: 0.7732 - val_loss: 2.9252 - val_accuracy: 0.7941
Epoch 3/5
845/845 [==============================] - 932s 1s/step - loss: 2.3523 - accuracy: 0.8293 - val_loss: 2.7690 - val_accuracy: 0.8181
Epoch 4/5
845/845 [==============================] - 923s 1s/step - loss: 1.9449 - accuracy: 0.8560 - val_loss: 3.1354 - val_accuracy: 0.8166
Epoch 5/5
845/845 [==============================] - 924s 1s/step - loss: 1.7029 - accuracy: 0.8714 - val_loss: 3.5834 - val_accuracy: 0.8148


In [34]:
accr = model.evaluate(X_test,Y_test1)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

209/209 [==============================] - 13s 60ms/step - loss: 3.2966 - accuracy: 0.8253
Test set
  Loss: 3.297
  Accuracy: 0.825


## CustomLLoss

In [45]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss=CustomLLoss(), optimizer='adam', metrics=['accuracy'])
epochs = 5
batch_size = 64
Y_train1 = tf.convert_to_tensor(Y_train, dtype=tf.float32)
Y_test1 = tf.convert_to_tensor(Y_test, dtype=tf.float32)
history = model.fit(X_train, Y_train1, epochs=epochs, batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
845/845 [==============================] - 915s 1s/step - loss: 1.5455 - accuracy: 0.6487 - val_loss: 1.5399 - val_accuracy: 0.7081
Epoch 2/5
845/845 [==============================] - 911s 1s/step - loss: 1.3357 - accuracy: 0.7796 - val_loss: 1.3054 - val_accuracy: 0.7984
Epoch 3/5
845/845 [==============================] - 911s 1s/step - loss: 1.2391 - accuracy: 0.8332 - val_loss: 1.2751 - val_accuracy: 0.8176
Epoch 4/5
845/845 [==============================] - 914s 1s/step - loss: 1.2025 - accuracy: 0.8523 - val_loss: 1.2936 - val_accuracy: 0.8163
Epoch 5/5
845/845 [==============================] - 914s 1s/step - loss: 1.1587 - accuracy: 0.8796 - val_loss: 1.2695 - val_accuracy: 0.8333


In [46]:
accr = model.evaluate(X_test,Y_test1)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

209/209 [==============================] - 12s 58ms/step - loss: 1.2625 - accuracy: 0.8426
Test set
  Loss: 1.263
  Accuracy: 0.843


## CustomLFullLoss

In [10]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss=CustomLFullLoss(), optimizer='adam', metrics=['accuracy'])
epochs = 5
batch_size = 64
Y_train1 = tf.convert_to_tensor(Y_train, dtype=tf.float32)
Y_test1 = tf.convert_to_tensor(Y_test, dtype=tf.float32)
history = model.fit(X_train, Y_train1, epochs=epochs, batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

2022-10-29 05:14:16.289805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-29 05:14:16.491733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-29 05:14:16.492673: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-29 05:14:16.494013: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/5
845/845 [==============================] - 953s 1s/step - loss: 0.6058 - accuracy: 0.6507 - val_loss: 0.4425 - val_accuracy: 0.7565
Epoch 2/5
845/845 [==============================] - 943s 1s/step - loss: 0.4041 - accuracy: 0.7710 - val_loss: 0.4467 - val_accuracy: 0.7448
Epoch 3/5
845/845 [==============================] - 942s 1s/step - loss: 0.2981 - accuracy: 0.8345 - val_loss: 0.3622 - val_accuracy: 0.7873
Epoch 4/5
845/845 [==============================] - 933s 1s/step - loss: 0.2544 - accuracy: 0.8546 - val_loss: 0.3254 - val_accuracy: 0.8271
Epoch 5/5
845/845 [==============================] - 898s 1s/step - loss: 0.2011 - accuracy: 0.8821 - val_loss: 0.3294 - val_accuracy: 0.8283


In [11]:
accr = model.evaluate(X_test,Y_test1)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

209/209 [==============================] - 13s 60ms/step - loss: 0.3066 - accuracy: 0.8394
Test set
  Loss: 0.307
  Accuracy: 0.839


In [ ]:
77, 77, 81, 83